In [1]:
import pandas as pd

!pip install thefuzz[levenshtein]

import thefuzz

from thefuzz import fuzz
from thefuzz import process

In [3]:
path_ipm = "C:\\Users\\63141\\Desktop\\github_files_v2\\unmatched_ipm_R2.xlsx"
ipm = pd.read_excel(path_ipm)

path_eia = "C:\\Users\\63141\\Desktop\\github_files_v2\\unmatched_eia_R2.xlsx"
eia = pd.read_excel(path_eia)

In [4]:
# Turning all letters in the data frame to lowercase so they can match easier 
eia = eia.applymap(lambda x: x.lower() if isinstance(x, str) else x)
ipm = ipm.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\63141\AppData\Local\Temp\ipykernel_7848\2246793782.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  eia = eia.applymap(lambda x: x.lower() if isinstance(x, str) else x)
C:\Users\63141\AppData\Local\Temp\ipykernel_7848\2246793782.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ipm = ipm.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Fuzzy match with pre processing 

In [6]:
import re

# Function to convert Roman numerals I through X to integers
def roman_to_int(roman):
    roman_numerals = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5,
        'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10,
        'i': 1, 'ii': 2, 'iii': 3, 'iv': 4, 'v': 5,
        'vi': 6, 'vii': 7, 'viii': 8, 'ix': 9, 'x': 10
    }
    return roman_numerals.get(roman, roman)

# Function to replace Roman numerals with integers in plant names
def replace_roman_numerals(name):
    roman_pattern = r'\b(I|II|III|IV|V|VI|VII|VIII|IX|X|i|ii|iii|iv|v|vi|vii|viii|ix|x)\b'
    matches = re.findall(roman_pattern, name)
    for match in matches:
        name = name.replace(match, str(roman_to_int(match)))
    return name

# Function to preprocess plant names
def preprocess_name(name):
    # Normalize to lower case
    name = name.lower()
    # Replace Roman numerals with integers
    name = replace_roman_numerals(name)
    # Remove special characters and common terms (e.g., 'wind', 'farm', etc.)
    name = name.replace('wind', '').replace('farm', '').replace('project', '').replace('energy', '').replace('solar','').replace('llc','').replace('center','').replace('no','').replace('hydro','').replace('station','').replace('generating','').replace('dam','').replace('power','').replace(
        'renewable','').replace('facility','').replace('plant','').replace('power','').replace('electric','')
    name = ''.join(e for e in name if e.isalnum() or e.isspace())
    return name.strip()

# Fuzzy matching and merge function
def fuzzy_merge(df1, df2, key1, key2, state_key, technology_key, threshold=85, limit=1):
    """
    Perform fuzzy matching and merge between two DataFrames based on plant names,
    but only if the state and technology type match. Each plant name can only match once.

    df1, df2: DataFrames to merge
    key1, key2: Column names for plant names to merge on
    state_key: Column name for states to match
    technology_key: Column name for technology to match
    threshold: Minimum similarity score to consider a match
    limit: Number of matches to return (best match)
    """
    matched_names = set()
    used_names = set()  # To keep track of names already used in matches

    # Preprocess plant names in both DataFrames
    df1['Cleaned Name'] = df1[key1].apply(preprocess_name)
    df2['Cleaned Name'] = df2[key2].apply(preprocess_name)

    def match_row(row):
        # Filter df2 based on matching state and technology type
        filtered_df2 = df2[
            (df2[state_key] == row[state_key]) &
            (df2[technology_key] == row[technology_key])
        ]

        # Apply fuzzy matching on the filtered DataFrame
        if not filtered_df2.empty and pd.notnull(row['Cleaned Name']):
            s = filtered_df2['Cleaned Name'].tolist()
            # Filter out names already used in previous matches
            s = [name for name in s if name not in used_names]
            if not s:
                return pd.Series([None, None], index=['best_match', 'match_score'])

            best_match = process.extractOne(row['Cleaned Name'], s, scorer=fuzz.token_set_ratio)
            if best_match and best_match[1] >= threshold:
                best_match_name = best_match[0]
                used_names.add(best_match_name)  # Mark this name as used
                return pd.Series([best_match_name, best_match[1]], index=['best_match', 'match_score'])

        return pd.Series([None, None], index=['best_match', 'match_score'])

    # Apply matching function
    match_results = df1.apply(match_row, axis=1)

    # Rename columns to avoid overlap
    match_results.columns = ['match_best_match', 'match_match_score']

    # Append match results to df1
    df1 = df1.join(match_results)

    # Perform the merge based on the best matches found
    merged_df = pd.merge(df1, df2, left_on='match_best_match', right_on='Cleaned Name', how='left', suffixes=('', '_df2'))

    # Drop the 'match_best_match' column if it exists
    if 'match_best_match' in merged_df.columns:
        merged_df = merged_df.drop(columns=['match_best_match'])

    return merged_df

# Example usage with your DataFrames:
merged_ipm_eia = fuzzy_merge(eia, ipm, 
                              'Plant Name', 'Plant Name', 
                              'State', 'Merged Capacity Types', 
                              threshold=85)

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


In [8]:
MS_90 = merged_ipm_eia[merged_ipm_eia['match_match_score'] >= 90]



In [9]:
path1 = "C:\\Users\\63141\\Desktop\\github_files_v2\\GMR3.xlsx"
MS_90.to_excel(path1, index= False)

In [11]:
final_df = ipm[~ipm['key'].isin(MS_90['key_df2'])]


In [13]:
path = "C:\\Users\\63141\\Desktop\\github_files_v2\\final_df.xlsx"
final_df.to_excel(path, index = False)